# Добавляем нужные библиотеки

In [1]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

import warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [2]:
dlgs=pd.read_csv('test_data.csv')
dlgs.head(15)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
6,0,6,client,Да мы работаем с компанией которая нам подлива...
7,0,7,client,Как как бы уже до этого момента работаем все у...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
9,0,9,client,Как бы нет


# Загружаем список русских имен, деловых приветствий и прощаний чтобы искать их в диалогах

In [3]:
def rus_names(file_name):
    names=[]
    
    with open(file_name, 'r', encoding='utf-8') as file:
        while True:
            line=file.readline()
            if not line: break
            names.append(line.replace('\n', '').lower())
            
    return names

In [4]:
names_list=rus_names('female_names_rus.txt')+rus_names('male_names_rus.txt')

greetings=['здравствуйте', 'добрый день', 'добрый вечер', 'доброе утро']
goodbye=['до свидания', 'до встречи', 'всего хорошего', 'всего доброго', 'хорошего дня', 'хорошего вечера']

# Делаем копию датасета для удобства (в копии делаем токенизацию текста)

In [5]:
dl_cop=dlgs.copy()

for i in range(len(dl_cop)):
    dl_cop['text'][i]=dl_cop['text'][i].lower().split()

# Сам скрипт

In [6]:
# количествао диалогов
dlgs_num=len(dlgs['dlg_id'].value_counts()) 

# создаем пустой список для важных столбцов нового датасета (имена менеджеров, компаний и т.д.)
dlg_id=[x for x in range(dlgs_num)]
managers_name=['' for x in range(dlgs_num)]
company=['' for x in range(dlgs_num)]
grt_repl=['' for x in range(dlgs_num)]
intro_repl=['' for x in range(dlgs_num)]
gb_repl=['' for x in range(dlgs_num)]
grt_gb_flag=[False for x in range(dlgs_num)]

# и делаем поиск в каждом диалоге
for i in range(dlgs_num):
    
    # чтобы знать здоровался и прощался ли менеджер, создаем 2 булевых переменных
    grts_flag=False
    gb_flag=False
    
    # ищем в начальных 3-х строках менеджера нужные данные (чаще всего приветствия и представления бывают в начальных 3-х строках)
    for j in range(3):
        
        # ищем в словах менеджера приветствие
        for gr in greetings:
            if not grts_flag and gr in dlgs['text'][dlgs['role']=='manager'][dlgs['dlg_id']==i].iloc[j].lower():
                grt_repl[i] = dlgs['text'][dlgs['role']=='manager'][dlgs['dlg_id']==i].iloc[j]
                grts_flag=True
        
        # ищем сначала представления в деловом стиле
        if re.findall(r'меня.+зовут | это | я ', dlgs['text'][dlgs['role']=='manager'][dlgs['dlg_id']==i].iloc[j].lower()):
            for word in dl_cop['text'][dl_cop['role']=='manager'][dl_cop['dlg_id']==i].iloc[j]:
                
                # если они есть то ищем в этой строке имена
                if word in names_list:
                    managers_name[i]=word[0].upper()+word[1:]
                    intro_repl[i]=dlgs['text'][dlgs['role']=='manager'][dlgs['dlg_id']==i].iloc[j]
            
        # ищем слово "компания", потому что в предствалениях компании всегда используется это слово
        if 'компания' in dl_cop['text'][dl_cop['role']=='manager'][dl_cop['dlg_id']==i].iloc[j]:
            
            # после мы создаем биграммы и триграммы используя эти строки
            biagram_collocation = BigramCollocationFinder.from_words(dl_cop['text'][dl_cop['role']=='manager'][dl_cop['dlg_id']==i].iloc[j])
            bg = biagram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10)
            
            trigram_collocation = TrigramCollocationFinder.from_words(dl_cop['text'][dl_cop['role']=='manager'][dl_cop['dlg_id']==i].iloc[j])
            tg=trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio, 20)
            
            
            # и ищем те биграммы и триграммы где слово "компания" стоит на первой позиции
            for row in bg:
                if 'компания' == row[0]:
                    bg_i=bg.index(row)
                
            for row in tg:
                if 'компания' == row[0]:
                    tg_i=tg.index(row)
                    
            # и сравниваем биграммы и триграммы, если библиотека сочла биграмму более подходящей чем триграмму то используем ее
            # в качестве названии компании, а если наоборот то триграмму
            if tg_i>bg_i:
                company[i] = ' '.join(bg[bg_i])
            else:
                company[i] = ' '.join(tg[tg_i])
            
            
        # ищем в последних трех строк менеджера реплики прощания
        for word in goodbye:
            if not gb_flag and word in dlgs['text'][dlgs['role']=='manager'][dlgs['dlg_id']==i].iloc[-j].lower():
                gb_repl[i] = dlgs['text'][dlgs['role']=='manager'][dlgs['dlg_id']==i].iloc[-j]
                gb_flag=True
        
            
        
                    
    # если менеджер и здоровался, и прощался, то grt_gb_flag будет иметь значение True, иначе False
    if grts_flag and gb_flag:
        grt_gb_flag[i] = True
    if not grts_flag:
        grt_repl[i]='Менеджер не поздаровался'
    if not gb_flag:
        gb_repl[i]='Менеджер не попращался'

In [7]:
df=pd.DataFrame()

df['dlg_id']=dlg_id
df['grt_repl']=grt_repl
df['managers_name']=managers_name
df['company']=company
df['intro_repl']=intro_repl
df['gb_repl']=gb_repl
df['grt_gb_flag']=grt_gb_flag

df

,dlg_id,grt_repl,managers_name,company,intro_repl,gb_repl,grt_gb_flag
0,0,Алло здравствуйте,Ангелина,компания диджитал бизнес,Меня зовут ангелина компания диджитал бизнес з...,Всего хорошего до свидания,True
1,1,Алло здравствуйте,Ангелина,компания диджитал бизнес,Меня зовут ангелина компания диджитал бизнес з...,До свидания,True
2,2,Алло здравствуйте,Ангелина,компания диджитал бизнес,Меня зовут ангелина компания диджитал бизнес з...,Менеджер не попращался,False
3,3,Алло дмитрий добрый день,Максим,компания китобизнес удобно,Добрый меня максим зовут компания китобизнес у...,Угу все хорошо да понедельника тогда всего доб...,True
4,4,Менеджер не поздаровался,,,,Во вторник все ну с вами да тогда до вторника ...,False
5,5,Менеджер не поздаровался,Анастасия,,Да это анастасия,Ну до свидания хорошего вечера,False
